<a href="https://colab.research.google.com/github/Yumnam1/AI-Assignment/blob/main/BERT_PEFT_LoRA_Fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import transformers

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from peft import get_peft_model,LoraConfig

In [6]:
lora_config = LoraConfig(r=16,
                         lora_alpha=32,
                         lora_dropout=0.1,
                         task_type = 'SEQ_CLS')

In [7]:
peft_model = get_peft_model(model,lora_config)

In [8]:
peft_model.train()

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default

In [10]:
text = ["I love the product","I hate the product","Great product","Bad product"]
labels = [1, 0, 1, 0]
encoded_text = tokenizer(text,return_tensors='pt',padding=True,truncation=True,max_length=128)
encoded_text

{'input_ids': tensor([[ 101, 1045, 2293, 1996, 4031,  102],
        [ 101, 1045, 5223, 1996, 4031,  102],
        [ 101, 2307, 4031,  102,    0,    0],
        [ 101, 2919, 4031,  102,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 0, 0]])}

In [11]:
import torch
label_ids = torch.tensor(labels)
optimizer = torch.optim.AdamW(peft_model.parameters())
for epoch in range(10):
  output = peft_model(input_ids=encoded_text['input_ids'],attention_mask=encoded_text['attention_mask'],labels=label_ids)
  loss = output.loss
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print(f"Epoch : {epoch+1} ========> Loss: {loss.item()}")

Epoch : 1 ========> Loss: 0.6710120439529419
Epoch : 2 ========> Loss: 0.6240019798278809
Epoch : 3 ========> Loss: 0.704657793045044
Epoch : 4 ========> Loss: 0.708246111869812
Epoch : 5 ========> Loss: 0.6199201345443726
Epoch : 6 ========> Loss: 0.5619661808013916
Epoch : 7 ========> Loss: 0.34205305576324463
Epoch : 8 ========> Loss: 0.28640711307525635
Epoch : 9 ========> Loss: 0.17476490139961243
Epoch : 10 ========> Loss: 0.10951816290616989


In [19]:
peft_model.eval()
input = "Awesome product"
encoded_input = tokenizer(input,return_tensors='pt',padding=True,truncation=True,max_length=128)
result = peft_model(input_ids=encoded_input['input_ids'],attention_mask=encoded_input['attention_mask'])

final_result = torch.argmax(result.logits,dim=1).item()
final_label = {0:"Negative",1:"Positive"}
print(f"Predicted {input} >>>>>>>>>>>>>>>> Result: {final_label[final_result]}")

Predicted Awesome product >>>>>>>>>>>>>>>> Result: Positive
